In [12]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
%matplotlib inline

In [13]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [14]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SECRET')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [15]:
INSTITUTION_ID = "ins_109508"

In [20]:
client = plaid.Client(
    client_id='5ea342792db15200132858',#PLAID_CLIENT_ID,
    secret='00d80844b0074c67355e20b6910d75',#PLAID_SECRET,
    public_key='3d7788c01e9a9c061ad3cf6b30e3ab',#PLAID_PUBLIC_KEY,
    environment='sandbox',
    api_version='2019-05-29'
)

### 2. Generate a public token

In [22]:
create_response = client.Sandbox.public_token.create(
    INSTITUTION_ID, ['transactions'])

### 3. Exchange the public token for an access token

In [28]:
exchange_response = client.Item.public_token.exchange(create_respopublic_token'])

access_token = exchange_response['access_token']

NameError: name 'public_token' is not defined

### 4. Fetch Accounts

In [27]:
response = client.Accounts.get(access_token)

NameError: name 'access_token' is not defined

In [ ]:
accounts = response['accounts']
pretty_print_response(accounts[0])

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [ ]:
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))

In [ ]:
response = client.Transactions.get(
    access_token,
    start_date = start_date,
    end_date = end_date,
)

In [ ]:
transactions = response['transactions']
pretty_print_response(transactions[0])

### 2. Print the categories for each transaction

In [ ]:
for transaction in transactions:
    print(transaction['category'][0])

### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [ ]:
list = []

for transaction in transactions:
    
    transaction_dict = {}

    transaction_dict['date'] = transaction['date']
    transaction_dict['name'] = transaction['name']
    transaction_dict['amount'] = transaction['amount']
    transaction_dict['category'] = transaction['category'][0]

In [ ]:
    list.append(transaction_dict)

In [ ]:
df = pd.DataFrame(list, columns=['date', 'name', 'amount', 'category'])
df.head()

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [ ]:
df['amount'].astype('float64')
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df.set_index('date', inplace=True, drop=True)
df.head()

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [ ]:
response = client.Income.get(
    access_token,
)

income = response['income']
pretty_print_response(income)

In [ ]:
print("last year's gross salary was", income['last_year_income_before_tax'])

In [ ]:
print('current monthly income is', income['income_streams'][0]['monthly_income'])

In [ ]:
print ('projected yearly income is', income['projected_yearly_income'])
print ('projected yearly income before tax is', income['projected_yearly_income_before_tax'])

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
total_spending = df.groupby('category').sum()
ax = total_spending.plot(y='amount', kind='pie', figsize=(10,10))

### Calculate the expenses per month

In [ ]:
monthly_expenses = df['amount'].resample('M').sum()
ax = monthly_expenses.plot(kind='bar')